#### 1) Запросить курсы любой валюты за последний год. Сохранить полученные данные в базу данных Mongo.  
#### 2) Написать функцию, которая принимает в качестве параметров две даты, ищет самую большую разницу между курсами валюты за указанный период и возвращает эти дни.  
#### 3) Сделать вывод информативным: Валюту <Валюта> выгодно было купить <Дата>, а продать <Дата>. Прибыль: <Разница в цене валюты>

In [1]:
import zeep
import pprint
import json
import datetime
from pymongo import MongoClient

In [2]:
url = "https://www.cbr.ru/DailyInfoWebServ/DailyInfo.asmx?WSDL"
zeepclient = zeep.Client(url)

Forcing soap:address location to HTTPS
Forcing soap:address location to HTTPS


In [3]:
mongoclient = MongoClient('mongodb://127.0.0.1:27017')
db = mongoclient['currencies']
currdb = db.currencies

In [4]:
def date_to_seconds(date):
    """
    Date example:
        date = '2018-12-30'
    """
    return int(datetime.datetime.timestamp(datetime.datetime.fromisoformat(date)))

def seconds_to_date(seconds):
    """
    Seconds example:
        seconds = 1576083600.0
    """
    return datetime.datetime.fromtimestamp(seconds).strftime('%Y-%m-%d')

def get_today_seconds():
    return int(datetime.datetime.timestamp(datetime.datetime.today()))

In [5]:
def date_curs(date):
    """
    Date example:
        date = '2018-12-30'
    """
    money = zeepclient.service.GetCursOnDate(date)
    new_valute_dict = {}
    for rec in money._value_1._value_1:
        key = rec["ValuteCursOnDate"]["VchCode"]
        value = {
            "Vnom": float(rec["ValuteCursOnDate"]["Vnom"]),
#             "Vcurs": {date: float(rec["ValuteCursOnDate"]["Vcurs"])},
            "Vcurs": {str(date_to_seconds(date)): float(rec["ValuteCursOnDate"]["Vcurs"])},
            "Vname": rec["ValuteCursOnDate"]["Vname"].rstrip()
        }
        new_valute_dict.update({key: value})
    return new_valute_dict

In [6]:
def show_what_in_database(print_limit=0):
    cursor = currdb.find()
    i = 0
    for document in cursor:
        pprint.pprint(document)
        print()
        i += 1
        if print_limit != 0:
            if i == print_limit:
                return

In [7]:
# Начальная дата, с которой будут добавлены все курсы валют (Формат: '2016-12-31')
START_DATE = "2015-12-31"

In [8]:
start_seconds = date_to_seconds(START_DATE)
today_seconds = get_today_seconds()
today_date = seconds_to_date(today_seconds)
print("Сегодня:", today_date)
seconds_in_one_day = 3600 * 24
print("Секунд в сутках:", seconds_in_one_day)

Сегодня: 2019-07-11
Секунд в сутках: 86400


In [9]:
# Drop database:
mongoclient.drop_database("currencies")

In [12]:
# for tmp_seconds in range(start_seconds, today_seconds, seconds_in_one_day):
for tmp_seconds in range(start_seconds, start_seconds + (30 * seconds_in_one_day), seconds_in_one_day):
    tmp_date = seconds_to_date(tmp_seconds)
    tmp_curs = date_curs(tmp_date)
    print(tmp_date)
    for valute in tmp_curs:
#         if currdb.find({valute: {"$exists": True}}).limit(1).count_documents() == 0:
        if valute == "KGS":
            print(tmp_curs[valute])
            print()
        if currdb.count_documents({valute: { "$exists": True}}) == 0:
            currdb.insert_one({valute: tmp_curs[valute]})
#             print(valute, "added")
        else:
#             print(valute, currdb.count_documents({valute: { "$exists": True}}))
            vcurs_dict = currdb.find_one({valute: { "$exists": True}})[valute]['Vcurs']
            vnom = currdb.find_one({valute: { "$exists": True}})[valute]['Vnom']
#             Номинал иногда меняется, поэтому надо его выравнивать
            if vnom != tmp_curs[valute]['Vnom']:
                tmp_multiplier = tmp_curs[valute]['Vnom'] / vnom
#                 tmp_curs[valute]['Vcurs'][tmp_date] = tmp_curs[valute]['Vcurs'][tmp_date] / tmp_multiplier
                tmp_curs[valute]['Vcurs'][str(tmp_seconds)] = tmp_curs[valute]['Vcurs'][str(tmp_seconds)] / tmp_multiplier
#             vcurs_dict.update({tmp_date: tmp_curs[valute]['Vcurs'][tmp_date]})
            vcurs_dict.update({str(tmp_seconds): tmp_curs[valute]['Vcurs'][str(tmp_seconds)]})
            
            currdb.update_one({valute: { "$exists": True}}, 
                              {'$set': {'%s.Vcurs' % valute: vcurs_dict}}, upsert=True)

2015-12-31
{'Vnom': 100.0, 'Vcurs': {'1451494800': 94.8376}, 'Vname': 'Киргизский сом'}

2016-01-01
{'Vnom': 100.0, 'Vcurs': {'1451581200': 94.9237}, 'Vname': 'Киргизский сом'}

2016-01-02
{'Vnom': 100.0, 'Vcurs': {'1451667600': 94.9237}, 'Vname': 'Киргизский сом'}

2016-01-03
{'Vnom': 100.0, 'Vcurs': {'1451754000': 94.9237}, 'Vname': 'Киргизский сом'}

2016-01-04
{'Vnom': 100.0, 'Vcurs': {'1451840400': 94.9237}, 'Vname': 'Киргизский сом'}

2016-01-05
{'Vnom': 100.0, 'Vcurs': {'1451926800': 94.9237}, 'Vname': 'Киргизский сом'}

2016-01-06
{'Vnom': 100.0, 'Vcurs': {'1452013200': 94.9237}, 'Vname': 'Киргизский сом'}

2016-01-07
{'Vnom': 100.0, 'Vcurs': {'1452099600': 94.9237}, 'Vname': 'Киргизский сом'}

2016-01-08
{'Vnom': 100.0, 'Vcurs': {'1452186000': 94.9237}, 'Vname': 'Киргизский сом'}

2016-01-09
{'Vnom': 100.0, 'Vcurs': {'1452272400': 94.9237}, 'Vname': 'Киргизский сом'}

2016-01-10
{'Vnom': 100.0, 'Vcurs': {'1452358800': 94.9237}, 'Vname': 'Киргизский сом'}

2016-01-11
{'Vnom': 1

In [11]:
show_what_in_database()

{'AUD': {'Vcurs': {'1451494800': 53.1242,
                   '1451581200': 53.3701,
                   '1451667600': 53.3701,
                   '1451754000': 53.3701,
                   '1451840400': 53.3701,
                   '1451926800': 53.3701,
                   '1452013200': 53.3701,
                   '1452099600': 53.3701,
                   '1452186000': 53.3701,
                   '1452272400': 53.3701,
                   '1452358800': 53.3701,
                   '1452445200': 53.3701,
                   '1452531600': 52.9756,
                   '1452618000': 53.3241,
                   '1452704400': 53.7515,
                   '1452790800': 53.1605,
                   '1452877200': 52.937,
                   '1452963600': 52.937,
                   '1453050000': 52.937,
                   '1453136400': 54.3831,
                   '1453222800': 54.3046,
                   '1453309200': 54.4628,
                   '1453395600': 57.7198,
                   '1453482000': 56.5

                   '1453395600': 12.3046,
                   '1453482000': 11.8986,
                   '1453568400': 11.8986,
                   '1453654800': 11.8986,
                   '1453741200': 11.4736,
                   '1453827600': 12.0663,
                   '1453914000': 11.6099,
                   '1454000400': 11.3713},
         'Vname': 'Индийская рупия',
         'Vnom': 10.0},
 '_id': ObjectId('5d263f258776c5d67d4f7d96')}

{'KZT': {'Vcurs': {'1451494800': 21.5165,
                   '1451581200': 21.3821,
                   '1451667600': 21.3821,
                   '1451754000': 21.3821,
                   '1451840400': 21.3821,
                   '1451926800': 21.3821,
                   '1452013200': 21.3821,
                   '1452099600': 21.3821,
                   '1452186000': 21.3821,
                   '1452272400': 21.3821,
                   '1452358800': 21.3821,
                   '1452445200': 21.3821,
                   '1452531600': 21.2064,
         

                   '1453309200': 28.3285,
                   '1453395600': 29.8221,
                   '1453482000': 28.6018,
                   '1453568400': 28.6018,
                   '1453654800': 28.6018,
                   '1453741200': 27.6168,
                   '1453827600': 29.0726,
                   '1453914000': 28.0429,
                   '1454000400': 27.4352},
         'Vname': 'Узбекский сум',
         'Vnom': 1000.0},
 '_id': ObjectId('5d263f258776c5d67d4f7da4')}

{'UAH': {'Vcurs': {'1451494800': 30.463,
                   '1451581200': 30.352,
                   '1451667600': 30.352,
                   '1451754000': 30.352,
                   '1451840400': 30.352,
                   '1451926800': 30.352,
                   '1452013200': 30.352,
                   '1452099600': 30.352,
                   '1452186000': 30.352,
                   '1452272400': 30.352,
                   '1452358800': 30.352,
                   '1452445200': 30.352,
                   '1

                   '1453222800': 66.5504,
                   '1453309200': 68.3862,
                   '1453395600': 71.3937,
                   '1453482000': 68.2201,
                   '1453568400': 68.2201,
                   '1453654800': 68.2201,
                   '1453741200': 65.737,
                   '1453827600': 69.4466,
                   '1453914000': 66.8191,
                   '1454000400': 65.0994},
         'Vname': 'Японская иена',
         'Vnom': 100.0},
 '_id': ObjectId('5d263f258776c5d67d4f7dab')}

